# SQL pré-armazenado: stored procedures, triggers, views

Além de armazenar nossos dados em tabelas, os bancos de dados também podem armazenar código SQL na forma de vários tipos de objetos:

- **Stored procedures**: procedimentos escritos em SQL, executados através da chamada `CALL`.
- **Stored functions**: funções escritas em SQL e que podem ser usadas nas mesmas situações que uma função pré-definida seria usada, como `SUM()` ou `COUNT()`
- **Triggers**: Um procedimento que será executado automaticamente quando determinadas condições ocorrem, como `INSERT`, `UPDATE` ou `DELETE` em uma tabela.
- **Eventos**: Procedimentos que podem ser executados pelo banco de dados em horários pré-definidos.
- **Views**: Diferente dos outros objetos, uma view é como um `SELECT` pré-definido, e resulta em uma tabela virtual.

Vamos continuar trabalhando com a base de dados **emprestimo** de uma aula anterior.

<img src="img/diagrama.png">

Para facilitar a atividade de hoje, vamos começar do zero rodando o script `emprestimos.sql`:

```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);
```

Em seguida, vamos abrir a conexão com o banco de dados.

In [ ]:
# Se estiver com problemas com o "multi"
!python -m pip install mysql-connector-python==8.3.0

In [2]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
script = """
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);
"""

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
)

with connection.cursor() as cursor:
    try:
        for _ in cursor.execute(script, multi=True):
            pass
        cursor.execute("COMMIT")
    except Exception as e:
        print(e)
        cursor.execute("ROLLBACK")

connection.close()

In [4]:
def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            # print("Executando queries:")
            results = cursor.execute(query, args, multi=True)
            for i, result in enumerate(results):
                if result.with_rows:
                    print(f"Resultado query {i}:")
                    for line in result.fetchall():
                        print(line)
                else:
                    print(f"Query {i} executada!")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("emprestimos")

## Stored procedures

Existem algumas vantagens em procedimentos armazenados (stored procedures):

- **Reuso de código**: como em qualquer outra linguagem de programação, podemos construir stored procedures para definir tarefas rotineiras e reutilizá-las em vários workflows.
- **Segurança**: Ao invés de oferecer acesso direto à tabelas, podemos bloquear todos os acessos a tabelas e definir acesso à stored procedures! Assim, ao invés de permitir acesso à tabela *usuario*, podemos permitir acesso ao procedimento `saque()` ou `consulta_saldo()`
- **Desempenho**: ao invés de mandar vários comandos SQL para a base de dados toda vez que quisermos realizar determinada tarefa, podemos simplesmente chamar um procedimento armazenado, pois todo o código já está na base de dados. Ademais, a base de dados não precisa compilar o procedimento toda vez que este é chamado, basta compilar na primeira execução e manter o código compilado em um cache.

Existem também desvantagens:

- **Debugging**: pode ser difícil debugar um procedimento armazenado
- **Portabilidade**: a sintaxe de definição de stored procedures é raramente portável entre diferentes sistemas de gerenciamento de bancos de dados.
- **Separação entre dados e lógica de negócios**: com stored procedures estamos migrando parte da lógica de negócios para o banco de dados. Alterações na lógica agora demandam modificações (e manutenção) em partes distintas da sua aplicação, uma receita para dor de cabeça!
- **Aumento da carga de processamento no servidor**: servidores de banco de dados costumam ser otimizados para memória e largura de banda, não para processamento.

Ou seja use procedimentos armazenados com cautela!

### Interlúdio: o valor da experiência

Como você já deve ter percebido à essa altura do curso, você tem um camilho longo pela frente, desde conhecer um conceito novo até dominar o uso deste conceito com sabedoria - isso chama-se ganhar experiência! Não subestime o valor de trabalhar "nas trincheiras" aplicando seus conhecimentos novos e aprendendo com a experiência dos veteranos. Além disso, continue se aperfeiçoando - faça cursos para conhecer melhor a tecnologia, invente projetos para tentar novas ideias, assista vídeos no YouTube sobre melhores práticas, busque projetos (e empregos) onde você possa se desenvolver.

É impossível formar engenheiros experientes em um curso - mesmo que fossem dois cursos, não cobriríamos tudo que existe sobre banco de dados. Mas estou certo de que vocês adquiriram novas ferramentas neste módulo do curso, e estão capacitados a continuar progredindo em banco de dados.

Nós, os professores do Insper, estamos fazendo o possível para transferir a vocês mais do que conhecimento - estamos buscando desenvolver competência também! E para isso só existe um caminho: a prática. Façam exercícios, coloquem esforço nos projetos, e venham preparados para a aula para que possamos ir além da mera leitura do material didático, ok?

Voltamos agora à programação regular...

### Construindo uma stored procedure

Vamos começar com uma stored procedure para adicionar um usuário. Crie e execute o seguinte script no MySQL workbench:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

DELIMITER //
CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END//
DELIMITER ;
```

Note o uso destes comandos `DELIMITER`. Coisas de MySQL: ele não entende que os ponto-e-virgula internos ao procedimento não sinalizam o final do comando CREATE PROCEDURE... Este procedimento será necessário quando quiser criar as procedures no Workbench, mas não deverá ser utilizado quando quiser rodar direto do python (com mysql.connector podemos utilizar ponto e vírgula).

Chamando pelo Python ficará assim:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END;
```

Agora podemos usar nossa nova procedure para adicionar alguns usuários!

In [7]:
db("""USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;


CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END;""")

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [8]:
db("SELECT * FROM usuario")

Resultado query 0:


In [9]:
db("COMMIT");

Query 0 executada!


In [10]:
db("START TRANSACTION;")
try:
    db("CALL adiciona_usuario('Juca', 'Silva');")
    db("CALL adiciona_usuario('Mario', 'Ferreira');")
    db("CALL adiciona_usuario('Ana', 'Soares');")
    db("CALL adiciona_usuario('Antonio', 'Reis');")
    db("CALL adiciona_usuario('Paulo', 'Oliveira');")
    db("COMMIT;")
except Exception as e:
    print(e)
    db("ROLLBACK");

Query 0 executada!
Query 0 executada!
Query 0 executada!
Query 0 executada!
Query 0 executada!
Query 0 executada!
Query 0 executada!


Vamos verificar se tudo funcionou bem:

In [11]:
db("SELECT * FROM usuario")

Resultado query 0:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


### Stored functions

Agora vamos criar uma função para consultar o saldo. Rode o script a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);

    SELECT 
        IFNULL(saldo, 0.0)
    INTO saldo_procurado FROM
        usuario
    WHERE
        id_usuario = id;

    RETURN saldo_procurado;
END//
DELIMITER ;
```

Vamos testar esta função informando o id de usuário `1`:

In [12]:
db("SELECT saldo(1)")

Resultado query 0:
(Decimal('0.00'),)


### Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [13]:
ia.tasks()

|    | Atividade            | De                  | Até                 | Conta como ATV?   | % Nota Atraso   |
|---:|:---------------------|:--------------------|:--------------------|:------------------|:----------------|
|  0 | newborn              | 2025-08-11 00:00:00 | 2025-11-30 00:00:00 | Não               | 0%              |
|  1 | select01             | 2025-08-13 00:00:00 | 2025-08-22 23:59:59 | Sim               | 25%             |
|  2 | ddl                  | 2025-08-25 00:00:00 | 2025-08-31 23:59:59 | Sim               | 25%             |
|  3 | dml                  | 2025-08-27 00:00:00 | 2025-09-04 23:59:59 | Sim               | 25%             |
|  4 | agg_join             | 2025-09-01 07:00:00 | 2025-09-07 23:59:59 | Sim               | 25%             |
|  5 | group_having         | 2025-09-03 07:00:00 | 2025-09-11 23:59:59 | Sim               | 25%             |
|  6 | views                | 2025-09-08 07:30:00 | 2025-09-14 23:59:59 | Sim               | 25%             |
|  7 | sql_review1          | 2025-09-11 07:30:00 | 2025-09-18 23:59:59 | Sim               | 25%             |
|  8 | permissions          | 2025-09-17 07:30:00 | 2025-09-25 23:59:59 | Sim               | 25%             |
|  9 | ai_md_23_1           | 2025-09-22 07:30:00 | 2025-09-30 23:59:59 | Sim               | 25%             |
| 10 | ai_md_23_2           | 2025-09-22 07:30:00 | 2025-09-30 23:59:59 | Sim               | 25%             |
| 11 | ai_md_25_2           | 2025-10-01 00:00:00 | 2025-10-01 12:10:00 | Não               | 0%              |
| 12 | desafio_normalizacao | 2025-10-06 14:15:00 | 2025-11-12 23:59:59 | Não               | 25%             |
| 13 | triggers             | 2025-10-20 07:30:00 | 2025-10-26 23:59:59 | Sim               | 25%             |

In [14]:
ia.grades(task="triggers")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | triggers    | ex01        |      1 |      0 |                 0 |                 0 |
|  1 | triggers    | ex02        |      1 |      0 |                 0 |                 0 |
|  2 | triggers    | ex04        |      1 |      0 |                 0 |                 0 |
|  3 | triggers    | ex05        |      1 |      0 |                 0 |                 0 |
|  4 | triggers    | ex06        |      1 |      0 |                 0 |                 0 |
|  5 | triggers    | ex07        |      1 |      0 |                 0 |                 0 |
|  6 | triggers    | ex08        |      1 |      0 |                 0 |                 0 |
|  7 | triggers    | ex09        |      1 |      0 |                 0 |                 0 |

In [15]:
ia.grades(by="task", task="triggers")

|    | Tarefa   |   Nota | Conta como ATV?   |
|---:|:---------|-------:|:------------------|
|  0 | triggers |      0 | Sim               |

**<div id="perg_atv1"></div>**
### Exercícios

<div class="alert alert-info">
    
**OBS**: Alteramos a função `db()` para aceitar o envio de múltiplas queries na mesma string!

</div>

#### Exercício 1
Faça um script SQL que cria uma stored procedure para cobrar uma taxa de manutenção de `D` dinheiros de cada conta, desde que a conta tenha saldo (não cobra de quem não tem saldo suficiente para descontar a taxa).

**Dica**: use a função `IF()`: https://dev.mysql.com/doc/refman/8.0/en/flow-control-functions.html#function_if

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Faça a partir do código base:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    -- Insira seu código aqui.
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
```   

</div>

In [16]:
sql_ex01 = """
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    UPDATE usuario
    SET saldo = IF(saldo >= taxa,
                        saldo - taxa,
                        saldo);
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
"""

db(sql_ex01)

Query 0 executada!
Query 1 executada!
Query 2 executada!


Escreva um código SQL para testar sua procedure.

Pense em como testar sua procedure. Se não conseguir, <a href="" title="1) Insira pelo menos três usuários, com saldo acima, abaixo e igual a taxa 2) Selecione (SELECT) os usuários 3) Chame a procedure para cobrar a taxa 4) Selecione os usuários e confira se o saldo foi atualizado.">pare o mouse aqui para dicas!</a>

**Resposta:**

In [17]:
db("UPDATE usuario SET saldo = 100.0 WHERE id_usuario = 1;")
db("CALL cobra_taxa(10);")
db("SELECT * FROM usuario WHERE id_usuario = 1;")

Query 0 executada!
Query 0 executada!
Resultado query 0:
(1, 'Juca', 'Silva', Decimal('90.00'))


In [18]:
ia.sender(answer="sql_ex01", task="triggers", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

<a href="#gab_atv1">Click para ver a resposta</a>

**<div id="perg_atv2"></div>**
#### Exercício 2

Faça uma função que retorna a soma de todos os saldos da base de dados.

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    -- Insira seu SQL aqui.

END;
```

</div>

In [5]:
sql_ex02 = """
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    DECLARE total DECIMAL(30, 2);
    SELECT SUM(saldo) INTO total FROM usuario;
    RETURN total;

END;
"""

db(sql_ex02)

Query 0 executada!
Query 1 executada!
Query 2 executada!


Escreva um código SQL para testar sua função. Para um teste mais efetivo, garanta que existem usuários com saldos variados na base de dados.

**Resposta:**

In [ ]:
db("-- Chame a function AQUI!")

In [6]:
ia.sender(answer="sql_ex02", task="triggers", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

<a href="#gab_atv2">Click para ver a resposta</a>

## Triggers

*Triggers*, ou gatilhos, são procedimentos armazenados que são executados automaticamente quando uma operação é realizada. Você pode escolher se o trigger ocorre antes ou depois da ação.

Vamos fazer um gatilho para atualizar automaticamente o saldo do usuário quando uma nova movimentação é inserida na tabela `movimentacao`. Vamos também criar uma *constraint* para impedir que um saldo negativo exista na nossa base de dados.

Bom, rode o script SQL a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;
```

*Interlúdio*: e quanto ao constraint `CHECK`? No MySQL 5.7 o `CHECK` não funcionava, apesar de previsto no padrão SQL. Na versão 8.0 isso foi corrigido.

Na documentação do MySQL 5.7 (https://dev.mysql.com/doc/refman/5.7/en/create-table.html) temos o seguinte:

    The `CHECK` clause is parsed but ignored by all storage engines. See Section 1.8.2.3, “Foreign Key Differences”.

A título de curiosidade: o remédio era criar um trigger para evitar a atualização com valores errados. Veja a gambiarra em ação no código a seguir.

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_saldo_insuficiente;

DELIMITER //
CREATE TRIGGER trig_saldo_insuficiente 
BEFORE UPDATE ON usuario
FOR EACH ROW
BEGIN
    -- https://stackoverflow.com/questions/2115497/check-constraint-in-mysql-is-not-working
    IF NEW.saldo < 0.0 THEN
        SIGNAL SQLSTATE '12345'
            SET MESSAGE_TEXT = 'Saldo insuficiente.';
    END IF;
END//
DELIMITER ;
```

Já na versão atual do MySQL o check de saldo pode ser implementado normalmente com o constraint `CHECK`

```SQL
USE emprestimos;
ALTER TABLE usuario ADD CONSTRAINT chk_saldo CHECK (saldo >= 0.0);
ALTER TABLE emprestimo ADD CONSTRAINT chk_emprestimo CHECK (valor_atual >= 0.0);
```

Vamos testar esse trigger na nossa tabela:

In [7]:
def movimentacao(id_usuario, valor):
    db(
        "INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)",
        (id_usuario, valor),
    )
    db("SELECT * FROM usuario WHERE id_usuario=%s", (id_usuario,))

In [8]:
db("START TRANSACTION")
movimentacao(1, 1000)

Query 0 executada!
Query 0 executada!
Resultado query 0:
(1, 'Juca', 'Silva', Decimal('1000.00'))


In [9]:
db('SELECT * FROM movimentacao')

Resultado query 0:
(1, 1, Decimal('1000.00'), datetime.datetime(2025, 10, 20, 11, 43, 21))


In [10]:
db('SELECT * FROM usuario')

Resultado query 0:
(1, 'Juca', 'Silva', Decimal('1000.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


Parece ter funcionado... mas antes de executar o `COMMIT`, vamos testar o que acontece quando o saldo ficaria negativo:

In [11]:
movimentacao(1, -20000)

DatabaseError: 3819 (HY000): Check constraint 'c_saldo' is violated.

Opa, o constraint impediu o saque inválido! Excelente! Vamos reverter as ultimas mudanças antes de prosseguir:

In [14]:
db("ROLLBACK")

Query 0 executada!


### Exercício 3

Realize as seguintes movimentações (com COMMIT se tudo funcionou dessa vez, e ROLLBACK se falhar):

| id_usuario | valor |
|--|--|
| 1 | +10000 |
| 2 | +4000 |
| 3 | +15000 |

In [15]:
movimentacao(1, 10000)
movimentacao(2, 4000)
movimentacao(3, 15000)

Query 0 executada!
Resultado query 0:
(1, 'Juca', 'Silva', Decimal('10000.00'))
Query 0 executada!
Resultado query 0:
(2, 'Mario', 'Ferreira', Decimal('4000.00'))
Query 0 executada!
Resultado query 0:
(3, 'Ana', 'Soares', Decimal('15000.00'))


**Conferir resposta**:

In [16]:
db("SELECT * FROM movimentacao")

Resultado query 0:
(5, 1, Decimal('10000.00'), datetime.datetime(2025, 10, 20, 11, 45, 47))
(6, 2, Decimal('4000.00'), datetime.datetime(2025, 10, 20, 11, 45, 47))
(7, 3, Decimal('15000.00'), datetime.datetime(2025, 10, 20, 11, 45, 48))


In [17]:
db("SELECT * FROM usuario")

Resultado query 0:
(1, 'Juca', 'Silva', Decimal('10000.00'))
(2, 'Mario', 'Ferreira', Decimal('4000.00'))
(3, 'Ana', 'Soares', Decimal('15000.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


### Exercício 4

Crie um trigger chamada `trig_transferencias_emprestimo` para realizar operações de transferências monetárias no contexto de um empréstimo.

Quando o usuário realiza um pagamento ou saque da linha de crédito (a linha de crédito é o empréstimo), é inserida uma `operacao`, o que deve acionar a trigger deste exercício.

A trigger deve:

- Atualizar o valor atual do empréstimo
- Inserir as movimentações, conforme valor da operação

**Obs**: Considere que já existe a trigger que atualiza o saldo do usuário, após inserções de `movimentacao`.

In [22]:
sql_ex04 = """
USE emprestimos;

DROP TRIGGER IF EXISTS trig_transferencias_emprestimo;

CREATE TRIGGER trig_transferencias_emprestimo
AFTER INSERT ON operacao
FOR EACH ROW
BEGIN
    UPDATE emprestimo 
        SET valor_atual = valor_atual + NEW.valor
        WHERE id_emprestimo = NEW.id_emprestimo;

    SELECT id_credor, id_devedor FROM emprestimo
    WHERE id_emprestimo = NEW.id_emprestimo
    INTO @id_credor, @id_devedor;

    INSERT INTO movimentacao(id_usuario, valor, data_operacao)
        VALUES(@id_credor, -NEW.valor, NOW());

    INSERT INTO movimentacao(id_usuario, valor, data_operacao)
        VALUES(@id_devedor, NEW.valor, NOW());
END;"""

db(sql_ex04)

Query 0 executada!
Query 1 executada!
Query 2 executada!


In [23]:
ia.sender(answer="sql_ex04", task="triggers", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Conferir resposta**:

In [24]:
db("SELECT * FROM usuario")

Resultado query 0:
(1, 'Juca', 'Silva', Decimal('10000.00'))
(2, 'Mario', 'Ferreira', Decimal('4000.00'))
(3, 'Ana', 'Soares', Decimal('15000.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


In [25]:
db("INSERT INTO emprestimo (id_credor, id_devedor) VALUES (2, 3)")

Query 0 executada!


In [26]:
db("SELECT * FROM emprestimo")

Resultado query 0:
(1, 2, 3, Decimal('0.00'), datetime.datetime(2025, 10, 20, 12, 9, 36), datetime.datetime(2025, 10, 20, 12, 9, 36))


In [27]:
db("SELECT * FROM operacao")

Resultado query 0:


In [28]:
db("INSERT INTO operacao (id_emprestimo, valor) VALUES (1, 2000)")

Query 0 executada!


In [29]:
db("SELECT * FROM operacao")

Resultado query 0:
(1, 1, Decimal('2000.00'), datetime.datetime(2025, 10, 20, 12, 9, 37))


In [30]:
db("SELECT * FROM emprestimo")

Resultado query 0:
(1, 2, 3, Decimal('2000.00'), datetime.datetime(2025, 10, 20, 12, 9, 36), datetime.datetime(2025, 10, 20, 12, 9, 37))


In [31]:
db("SELECT * FROM usuario")

Resultado query 0:
(1, 'Juca', 'Silva', Decimal('10000.00'))
(2, 'Mario', 'Ferreira', Decimal('2000.00'))
(3, 'Ana', 'Soares', Decimal('17000.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


In [32]:
db("SELECT * FROM movimentacao")

Resultado query 0:
(5, 1, Decimal('10000.00'), datetime.datetime(2025, 10, 20, 11, 45, 47))
(6, 2, Decimal('4000.00'), datetime.datetime(2025, 10, 20, 11, 45, 47))
(7, 3, Decimal('15000.00'), datetime.datetime(2025, 10, 20, 11, 45, 48))
(8, 2, Decimal('-2000.00'), datetime.datetime(2025, 10, 20, 12, 9, 37))
(9, 3, Decimal('2000.00'), datetime.datetime(2025, 10, 20, 12, 9, 37))


## Relembrando: Views

*Views* são como tabelas *'lógicas'*, que são criadas através da aplicação de um comando `SELECT`. É como um `SELECT` pré-armazenado.

Vamos criar uma *view* para listar os nomes e sobrenomes de usuários, sem revelar seus saldos de conta:

```SQL
USE emprestimos;

CREATE VIEW nomes AS 
	SELECT DISTINCT nome, sobrenome FROM usuario;
```

Agora podemos usar esta view em consultas:

In [34]:
db("SELECT * FROM nomes;")

Resultado query 0:
('Juca', 'Silva')
('Mario', 'Ferreira')
('Ana', 'Soares')
('Antonio', 'Reis')
('Paulo', 'Oliveira')


Porque usar views? Assim como no caso de stored procedures, podemos configurar permissões de acesso diferentes para esta view. Suponha que um vendedor deva ter acesso aos nomes dos clientes, mas não aos seus saldos (por razões de confidencialidade). Podemos conceder ao vendedor acesso apenas à essa view. Poderíamos ter resolvido o problema também com uma stored procedure: em SQL as coisas costumam ter várias soluções possíveis...

### Exercício 5

Crie uma view chamada `credor` para mostrar, para cada usuário, a soma dos valores de emprestimo dos quais este cliente é credor. Se o cliente não tiver empréstimo associado como credor, o valor deve ser zero.

As colunas retornadas devem se chamar `id_usuario`, `nome`, `sobrenome` e `total`, onde `total` representa a soma dos valores de empréstimo dos quais o cliente é credor.

Ordene pelo `total` (decrescente), `nome` (crescente) e `sobrenome` (crescente).

In [36]:
sql_ex05 = """
CREATE VIEW credor AS
    SELECT
        id_usuario,
        nome,
        sobrenome,
        COALESCE(SUM(e.valor_atual), 0) AS total
    FROM
        usuario u
        LEFT JOIN emprestimo e ON u.id_usuario = e.id_credor
    GROUP BY
        id_usuario
    ORDER BY
        total DESC, nome ASC, sobrenome ASC
    
"""

db(sql_ex05)

Query 0 executada!


In [37]:
ia.sender(answer="sql_ex05", task="triggers", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 6

Repita o processo para a posição de devedor. A view deve se chamar `devedor`.

Considere os mesmos nomes de colunas e critérios de ordenação.

In [38]:
sql_ex06 = """
CREATE VIEW devedor AS
    SELECT
        id_usuario,
        nome,
        sobrenome,
        COALESCE(SUM(e.valor_atual), 0) AS total
    FROM
        usuario u
        LEFT JOIN emprestimo e ON u.id_usuario = e.id_devedor
    GROUP BY
        id_usuario
    ORDER BY
        total DESC, nome ASC, sobrenome ASC
"""

db(sql_ex06)

Query 0 executada!


In [39]:
ia.sender(answer="sql_ex06", task="triggers", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 7

Crie uma view `posicao_liquida` que lista o **valor líquido** de cada usuário, que é a soma dos valores como credor mais o saldo, subtraido da soma dos valores como devedor.

As colunas retornadas devem se chamar `id_usuario`, `nome`, `sobrenome` e `total_liquido`, onde `total_liquido` representa o **valor líquido**.

Ordene por `total_liquido` (decrescente), nome (crescente) e sobrenome (decrescente).

In [41]:
sql_ex07 = """
CREATE VIEW posicao_liquida AS
    SELECT
        u.id_usuario,
        u.nome,
        u.sobrenome,
        c.total + u.saldo - d.total AS total_liquido
    FROM
        usuario u
        INNER JOIN credor c USING(id_usuario)
        INNER JOIN devedor d USING(id_usuario)
    ORDER BY
        total_liquido DESC, u.nome ASC, u.sobrenome DESC
"""

db(sql_ex07)

Query 0 executada!


In [42]:
ia.sender(answer="sql_ex07", task="triggers", question="ex07", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex07', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 8

Crie uma função para calcular o preço com desconto.

A função se chama `aplica_desconto` e recebe, nesta ordem, o `preco DECIMAL(10,2)` e o `percentual DECIMAL(10,2)` de desconto (intervalo `[0,100]`).

**Dicas**:
- procure por *"MySQL functions DETERMINISTIC, NO SQL, READS SQL DATA"*.
- você pode criá-la em qualquer base de dados (não tem relação com a base `emprestimos`).

In [43]:
sql_ex08 = """
CREATE FUNCTION aplica_desconto(preco DECIMAL(10,2), percentual DECIMAL(10,2)) RETURNS DECIMAL(10, 2) DETERMINISTIC
BEGIN

    RETURN preco - preco * percentual / 100;

END;
"""

db(sql_ex08)

Query 0 executada!


In [44]:
ia.sender(answer="sql_ex08", task="triggers", question="ex08", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex08', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 9

Considere uma base chamada de **clientes** e **solicitações** chamada `solicita`:

<img src="img/solicita.png">

```mysql
DROP SCHEMA IF EXISTS `solicita` ;

CREATE SCHEMA IF NOT EXISTS `solicita` ;
USE `solicita` ;

DROP TABLE IF EXISTS `solicita`.`cliente` ;

CREATE TABLE IF NOT EXISTS `solicita`.`cliente` (
  `id_cliente` INT NOT NULL,
  `nome` VARCHAR(100) NOT NULL,
  `sobrenome` VARCHAR(150) NOT NULL,
  `cpf` CHAR(14) NULL,
  `ultimo_acesso` DATE NULL,
  PRIMARY KEY (`id_cliente`),
  UNIQUE INDEX `cpf_UNIQUE` (`cpf` ASC) VISIBLE)
ENGINE = InnoDB;


DROP TABLE IF EXISTS `solicita`.`solicitacao` ;

CREATE TABLE IF NOT EXISTS `solicita`.`solicitacao` (
  `id_solicitacao` INT NOT NULL,
  `id_cliente` INT NOT NULL,
  `mensagem` TEXT NOT NULL,
  `data` DATE NULL,
  PRIMARY KEY (`id_solicitacao`),
  INDEX `fk_venda_cliente_idx` (`id_cliente` ASC) VISIBLE,
  CONSTRAINT `fk_venda_cliente`
    FOREIGN KEY (`id_cliente`)
    REFERENCES `solicita`.`cliente` (`id_cliente`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;
```

Crie uma **procedure** que deleta **todos os dados** dos clientes cujo último acesso foi há mais que 180 dias.

A procedure deve se chamar `remove_clientes_inativos`.

**Dicas**:
- Como critério de último acesso, considere a coluna `ultimo_acesso` na tabela de `cliente`.
- Utilize os scripts SQL disponíveis na pasta de aulas para inserir clientes e solicitações na base e efetuar os testes!
- Altere as datas, pois os 180 dias são em relação a data atual.

In [58]:
sql_ex09 = """
CREATE PROCEDURE remove_clientes_inativos()
BEGIN
    DELETE FROM solicitacao
    WHERE id_cliente IN (
        SELECT id_cliente
        FROM cliente
        WHERE ultimo_acesso < CURDATE() - INTERVAL 180 DAY
    );

    DELETE FROM cliente
    WHERE ultimo_acesso < CURDATE() - INTERVAL 180 DAY;
END;
"""

db(sql_ex09)

Query 0 executada!


In [57]:
db("DROP PROCEDURE IF EXISTS remove_clientes_inativos")

Query 0 executada!


In [59]:
ia.sender(answer="sql_ex09", task="triggers", question="ex09", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex09', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Exercício 10

Este exercício não tem correção automática e sua resolução é opcional.

Considere uma versão base de **clientes** e **solicitações** onde os deletes são **lógicos** e não físicos:

<img src="img/solicita_v2.png">

Neste exercício, você deve:

- Criar procedure para **deletar logicamente clientes** cujo último acesso foi há mais que 180 dias. A procedure deve deletar logicamente apenas clientes e não solicitações.
- Criar trigger na tabela de cliente para que, ao deletar logicamente um cliente, suas solicitações também sejam deletadas.
- Criar trigger na tabela de solicitação para impedir a inserção de solicitações para cliente deletados.

In [ ]:
sql_ex10 = """
-- Seu SQL aqui!
"""

db(sql_ex10)

Por hoje é só!

In [60]:
connection.close()

### Conferir notas

In [3]:
ia.grades(by="task", task="triggers")

|    | Tarefa   |   Nota | Conta como ATV?   |
|---:|:---------|-------:|:------------------|
|  0 | triggers |     10 | Sim               |

In [4]:
ia.grades(task="triggers")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | triggers    | ex01        |      1 |     10 |                10 |                 0 |
|  1 | triggers    | ex02        |      1 |     10 |                10 |                 0 |
|  2 | triggers    | ex04        |      1 |     10 |                10 |                 0 |
|  3 | triggers    | ex05        |      1 |     10 |                10 |                 0 |
|  4 | triggers    | ex06        |      1 |     10 |                10 |                 0 |
|  5 | triggers    | ex07        |      1 |     10 |                10 |                 0 |
|  6 | triggers    | ex08        |      1 |     10 |                10 |                 0 |
|  7 | triggers    | ex09        |      1 |     10 |                10 |                 0 |

## Gabarito

**<div id="gab_atv1">Atividade 1</div>**
<div class="alert alert-warning">

Para Workbench - SEM **IF**:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    -- Insira seu código aqui.
    UPDATE usuario 
    SET 
        saldo = saldo - taxa
    WHERE
        saldo >= taxa;
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END //
DELIMITER ;
```

Para Workbench - COM **IF**:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;
    UPDATE usuario SET saldo = IF(saldo > taxa, saldo - taxa, saldo);
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END//
DELIMITER ;
```

Para Python - SEM **IF**:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    -- Gambiarra para desligar o aviso de segurança que previne
    -- UPDATES gerais (sem WHERE).
    -- OBS: da pra fazer sem ela!
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    -- Insira seu código aqui.
    UPDATE usuario 
    SET 
        saldo = saldo - taxa
    WHERE
        saldo >= taxa;
    
    -- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END;
```
</div>
<a href="#perg_atv1">Click para voltar ao Exercício</a>


**<div id="gab_atv2">Atividade 2</div>**
<div class="alert alert-warning">

Para Workbench:
```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

DELIMITER //
CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    -- Insira seu SQL aqui.
    DECLARE total DECIMAL(30, 2);
    SELECT SUM(saldo) INTO total FROM usuario;
    RETURN total;

END//
DELIMITER ;
```
</div>
<a href="#perg_atv2">Click para voltar ao Exercício</a>